<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèles mis en oeuvre : 
    * Multinomial Naive Bayes
    * Multinomial Naive Bayes chaîné
    * Regression logistic (GSCV)
    * SGD (GSCV)
    * Multinomial Bernoulli 

Ce notebook utilise les données issues des notebooks : 


**`P6_DadaAnalysis.ipynb`**

**`P6_UnsupervizedMethods.ipynb`**



In [126]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# <font color=blus>4. Supervized methods</font>

## <font color=blus>4.0. Encoding TAGs issued from train and test data-sets</font>

* Multiple TAGs are assigned to a unique POST.

    This lead to deal with a multi-label classification issue.

    All labels (TAGs) are one hot encoded.


* Train and test data-sets do not have same assigned TAGs set.

    This leads to a different number of encoded columns for each data-set.

    For having same number of columns for train and test datasets, encoding process uses the union of train and 
    test assigned TAGs .



In [1]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_test=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_test.shape)

file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_train=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_train.shape)



(16359,)
(24604,)


**<font color='blue'>List of unique TAGs is built</font>**

In [2]:
import p6_util

# String of TAGs are converted as list of TAGS.
list_list_tags  = ser_sof_tags_train.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags))
list_list_tags += ser_sof_tags_test.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags))
#list_ref_tags = list(set(list_ref_tags))
#print(len(list_ref_tags))
list_ref_tags = list()
for list_tags in list_list_tags :
    list_ref_tags += [tag for tag in list_tags]

list_ref_tags = list(set(list_ref_tags))
print(len(list_ref_tags))


24604
40963
8385


**<font color='blue'>TAGs from train and test datasets are encoded and saved into CSR matrix</font>**

In [3]:
import p5_util
import p6_util

csr_matrix_encoded_train_tags, list_ref_train_tags = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_train, list_ref_tags)
print("csr_matrix_encoded_train_tags : {}".format(csr_matrix_encoded_train_tags.shape))

file_name="./data/csr_matrix_encoded_train_tags.dump"
p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)

csr_matrix_encoded_train_tags : (24604, 8385)


In [4]:
import p5_util
import p6_util

csr_matrix_encoded_test_tags, list_ref_test_tags = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_test, list_ref_tags)
print("csr_matrix_encoded_test_tags : {}".format(csr_matrix_encoded_test_tags.shape))

file_name="./data/csr_matrix_encoded_test_tags.dump"
p5_util.object_dump(csr_matrix_encoded_test_tags,file_name)

csr_matrix_encoded_test_tags : (16359, 8385)


## <font color=blus>4.2. Test dataset processing</font>

Processing results in : 
    * POSTs standardization
    * POSTs vectorization (words embedded on dictionary issued from most weighted tokens).
    * TAGs from POSTs are encoded in a one hot way and saved into CSR matrix.

In [5]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)
print(df_sof_test.shape)

(16359, 5)


**<font color='blue'>Test dataset standardization</font>**


Process is applied on a POST that is built from merged `Body` and `Title` columns

In [6]:
df_sof_test['Body']=df_sof_test['Body']+df_sof_test['Title']

In [7]:
import p5_util
import p6_util

ser_sof_body_test = p6_util.p6_df_standardization(df_sof_test.Body)

file_name='./data/ser_sof_body_test.dump'
p5_util.object_dump(ser_sof_body_test, file_name)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Removing stopwords...

Lemmatization ...


**<font color='blue'>Test dataset Vectorization</font>**

* Loading TF-IDF vectorizer; this is same operator used for train data-set vectorization.

In [18]:
import p5_util
file_name="./data/csr_matrix_tfidf_ngram1_b.dump"
csr_matrix_tfidf_1gram_train = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tfidf_ngram1_b.dump"
vectorizer = p5_util.object_load(file_name)

p5_util.object_load : fileName= ./data/csr_matrix_tfidf_ngram1_b.dump
(24604, 4791)
p5_util.object_load : fileName= ./data/vectorizer_tfidf_ngram1_b.dump


In [10]:
import p6_util

csr_matrix_tdif_1gram_test = vectorizer.transform(ser_sof_body_test)
csr_matrix_tdif_1gram_test.shape

(16359, 4791)

## <font color=blus>4.3. Loading SOF TAGs</font>

Those TAGs from SOF database are used as targets for M.L. process.


In [11]:
import pandas as pd
file_name="./data/StackOverFlowTags.csv"
df_sof_tags=pd.read_csv(file_name)
df_sof_tags.shape

(50000, 2)

## <font color=blus>4.4. Loading vectorized train dataset and encoded train target dataset</font>

**Loading TF-IDF data-set containing trained vectorized POSTs**

In [12]:
import p5_util

file_name="./data/csr_matrix_tfidf_1gram.dump"
csr_matrix_tfidf_1gram = p5_util.object_load(file_name)
print(csr_matrix_tfidf_1gram.shape)

p5_util.object_load : fileName= ./data/csr_matrix_tfidf_1gram.dump
(16359, 3641)


**Loading encoded train target data-set**

In [16]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_tags.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_train_tags.dump
(24604, 8385)


## <font color=blus>4.5. Loading vectorized test dataset and encoded test target dataset</font>

**Loading TF-IDF data-set containing test vectorized POSTs**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_1gram_test.dump"
csr_matrix_tdif_1gram_test = p5_util.object_load(file_name)
csr_matrix_tdif_1gram_test.shape

**Loading encoded test target data-set**

In [ ]:
import p5_util

file_name="./data/csr_matrix_encoded_test_tags.dump"
csr_matrix_encoded_test_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_test_tags.shape)

In [13]:
dict_cls_score = dict()

In [19]:
X_train = csr_matrix_tfidf_1gram_train
y_train = csr_matrix_encoded_train_tags

X_test = csr_matrix_tdif_1gram_test
y_test = csr_matrix_encoded_test_tags

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

col_min = min(y_train.shape[1],  y_test.shape[1] )

print("\nMin col for target : \n"+str(col_min))

if False:
    y_train = y_train[:, :col_min]
    y_test  = y_test[:,:col_min]

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(24604, 4791) (24604, 8385) (16359, 4791) (16359, 8385)

Min col for target : 
8385


## <font color='blus'>4.6. Evaluation of Multinomial Bernouilli Naive Bayes classifier</font>

**<font color='blue'>Loading SOF TAGs </font>**

In [ ]:
import pandas as pd
file_name="./data/StackOverFlowTags.csv"
df_sof_tags=pd.read_csv(file_name)
df_sof_tags.shape

**<font color='blue'>Loading encoded test target data-set</font>**

In [ ]:
import p5_util

file_name="./data/csr_matrix_encoded_test_tags.dump"
csr_matrix_encoded_test_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_test_tags.shape)

**<font color='blue'>Loading TF-IDF test data-set containing vectorized POSTs</font>**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_1gram_test.dump"
csr_matrix_tdif_1gram_test = p5_util.object_load(file_name)
csr_matrix_tdif_1gram_test.shape

**<font color='blue'>Loading encoded train target data-set</font>**

In [ ]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_tags.shape)

**<font color='blue'>Loading TF-IDF train data-set containing vectorized POSTs</font>**

In [ ]:
import p5_util

file_name="./data/csr_matrix_tfidf_ngram1_b.dump"
csr_matrix_tfidf_ngram1_b = p5_util.object_load(file_name)
print(csr_matrix_tfidf_ngram1_b.shape)

In [ ]:
X_train = csr_matrix_tfidf_ngram1_b
y_train = csr_matrix_encoded_train_tags

X_test = csr_matrix_tdif_1gram_test
y_test = csr_matrix_encoded_test_tags


col_min = min(y_train.shape[1],  y_test.shape[1] )

print("\nMin col for target : "+str(col_min))

if False:
    y_train = y_train[:, :col_min]
    y_test  = y_test[:,:col_min]

print("\n *** X_train= {} y_train={} X_test={} y_test={}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))


In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
N_JOBS=-1
cls_ber = OneVsRestClassifier(BernoulliNB(), n_jobs=N_JOBS).fit(X_train, y_train)

In [ ]:
cls_ber

**Compute accuracy**

In [ ]:
y_pred = cls_ber.predict(X_test)

In [ ]:
import p6_util
import p5_util
cls_name = 'Bernoulli NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(csr_matrix_encoded_test_tags, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

## <font color=blus>4.7. Applying Naive Bayes classifier</font>

### <font color=blus>4.7.1. Training N.B. classifier</font>

**Training classifier**

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes. 

Then, one classifier per class is fitted.



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

cls_mnb = OneVsRestClassifier(MultinomialNB(), n_jobs=-1).fit(X_train, y_train)


In [ ]:
cls_mnb

### <font color='blus'>4.7.2.Classifier evaluation</font>

**TAGs predictions and probabilities for test dataset**

In [ ]:
y_pred = cls_mnb.predict(X_test)

In [ ]:
import p6_util
cls_mnb_score = p6_util.p6_supervized_mean_accuracy_score(csr_matrix_encoded_test_tags, y_pred)
print("Accuracy score for Multinomial NB : {0:1.2F} %".format(cls_mnb_score*100))

In [ ]:
cls_mnb_score

In [ ]:
import p6_util
import p5_util
cls_name = 'Multinomial NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(csr_matrix_encoded_test_tags, y_pred)
print("Accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

**<font color=blue>P({Tag_i}|Post)</font>**

In [ ]:
y_proba = model.predict_proba(X_test)

y_proba.shape

**Compute accuracy**

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = cls_mnb.predict(X_test)

cls_mnb_score = accuracy_score(y_test  ,y_pred)

print("Accuracy score for MNB classifier : {0:1.2F} %".format(cls_mnb_score*100))

dict_cls_score['MNB'] = cls_mnb_score

In [ ]:
import p5_util

file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score,file_name)


## <font color='blus'>4.8. Chained Multinomial Naive Bayes classifier</font>

In [ ]:
import p5_util

file_name="./data/dict_cls_score.dump"
dict_cls_score = p5_util.object_load(file_name)


In [ ]:
from skmultilearn.problem_transform import ClassifierChain

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

chained_classifier_mnb = ClassifierChain(MultinomialNB())

In [ ]:
chained_classifier_mnb.fit(X_train , y_train)

In [ ]:
y_pred = chained_classifier_mnb.predict(X_test)

In [ ]:
import p6_util
import p5_util
cls_name = 'Chained Multinomial NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(csr_matrix_encoded_test_tags, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

## <font color='blus'>4.9. SGD classifier</font>

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

ovr_sgd_classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)).fit(X_train, y_train)

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 10 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 14 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 20 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packa

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 199 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 202 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 210 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 223 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 457 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 469 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 470 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 474 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 476 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 477 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 678 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 684 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 688 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 694 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 707 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 710 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 926 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 934 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 937 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 938 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 940 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 950 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7379 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7382 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7389 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7392 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7395 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7400 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7581 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7582 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7588 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7596 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7601 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7604 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7778 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7793 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7794 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7795 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7800 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7802 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7983 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7988 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7994 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7996 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8002 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8004 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8217 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8219 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8220 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8222 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8225 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8245 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

In [31]:
import p5_util

file_name="./data/ovr_sgd_classifier.dump"
p5_util.object_dump(ovr_sgd_classifier,file_name )


In [33]:
print (ovr_sgd_classifier)
#print (ovr_sgd_classifier.best_params_)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None)


In [39]:
y_pred = sgd_classifier.predict(X_test)

In [40]:
import p6_util
import p5_util
cls_name = 'SGD'
cls_score = p6_util.p6_supervized_mean_accuracy_score(csr_matrix_encoded_test_tags, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

48055
Mean accuracy score for SGD : 0.00 %


{'SGD': 0.0, 'Linear Regression': 0.0010612839454791384}

## <font color='blus'>4.10. Grid Search CV Logistic regression & One vs rest</font>

In [ ]:
import p5_util
import p6_util

logreg_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2', C=10, n_jobs=-1))

logreg_classifier.fit(X_train, y_train)

y_pred = logreg_classifier.predict(X_test)

In [171]:
cls_name = 'Logistic Regression'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred, mode_match='fuzzy_matching')
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

p6_get_dict_row_col_from_csrmatrix
16359
p6_get_dict_row_col_from_csrmatrix
626
3918 [5454, 6265, 7968]


TypeError: cannot use a string pattern on a bytes-like object

In [157]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV

import p6_util

def gscv_best_classifier_result(dict_param_grid, classifier, X_train, y_train\
                              , cls_name, X_test,y_test, dict_cls_score):
    ovr_classifier = OneVsRestClassifier(classifier)
    gscv_classifier  = GridSearchCV(ovr_classifier, dict_param_grid)

    gscv_classifier.fit(X_train, y_train)
    print (gscv_classifier.best_score_)
    print (gscv_classifier.best_params_)
    y_pred = gscv_classifier.best_estimator_.predict(X_test)
    cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
    print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))
    dict_cls_score[cls_name]=cls_score
    return dict_cls_score, gscv_classifier.best_estimator_

    

In [164]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.A)
X_train_scaled = scaler.transform(X_train.A)


scaler.fit(X_test.A)
X_test_scaled = scaler.transform(X_test.A)


In [165]:
LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [166]:
import p5_util

dict_param_grid = {
    #'estimator__C': [ 25, 50,100],
    'estimator__C': [ 0.1],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}
cls_name='Logistic Regression'
dict_cls_score, best_classfier = gscv_best_classifier_result(dict_param_grid,LogisticRegression(), X_train_scaled, y_train\
                                               , cls_name, X_test_scaled, y_test, dict_cls_score)
file_name = './data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_best_logisticRegression.dump'
p5_util.object_dump(best_classfier, file_name)
dict_cls_score

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/l

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs

KeyboardInterrupt: 

In [80]:
import p5_util
file_name = './data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_best_logisticRegression.dump'
p5_util.object_dump(best_classfier, file_name)


In [132]:
file_name = './data/classifier_best_logisticRegression.dump'
best_classfier= p5_util.object_load(file_name)

p5_util.object_load : fileName= ./data/classifier_best_logisticRegression.dump


In [133]:
best_classfier

OneVsRestClassifier(estimator=LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=-1)

In [144]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import parfit.parfit as pf

def parfit_best_classifier_result(dict_param_grid, classifier, X_train, y_train\
                              , cls_name, X_test, y_test, dict_cls_score):
    paramGrid = ParameterGrid(dict_param_grid)
    
    bestModel, bestScore, allModels, allScores = pf.bestFit(classifier, paramGrid,\
           X_train, y_train, X_test, y_test, metric = roc_auc_score, \
           scoreLabel = "AUC")
    
    print(bestModel, bestScore)
    return bestModel


In [150]:
X_train.shape, y_train.shape

((24604, 4791), (24604, 8385))

## <font color='blus'>4.11. Grid search for SVM classifier with SGD</font>

In [158]:
from sklearn.linear_model import SGDClassifier
import p5_util
file_name = './data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_param_grid = {
    'estimator__loss':['hinge'],
    #'estimator__alpha': [1e-3, 5*1e-3, 1e-2, 5e-2, 1e-1,5e-1,1.,10,20],
    'estimator__alpha': [ 1e-1],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}

cls_name='SGD Classifier'
dict_cls_score, best_classfier = gscv_best_classifier_result(dict_param_grid, SGDClassifier(), X_train, y_train\
                                               , cls_name, X_test, y_test,dict_cls_score)


p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_bestSGD_SVM.dump'
p5_util.object_dump(best_classfier, file_name)
dict_cls_score

p5_util.object_load : fileName= ./data/dict_cls_score.dump


/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.0
{'estimator__alpha': 0.1, 'estimator__loss': 'hinge', 'estimator__penalty': 'l2', 'n_jobs': -1}
p6_get_dict_row_col_from_csrmatrix
16359
p6_get_dict_row_col_from_csrmatrix
0
48055
Mean accuracy score for SGD Classifier : 0.00 %


AttributeError: module 'p5_util' has no attribute 'objdump'

## <font color='blus'>4.12. Powered SGD classifier</font>

In [ ]:
from sklearn.linear_model import SGDClassifier
from skmultilearn.problem_transform import LabelPowerset
power_classifier_sgd = LabelPowerset(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))
power_classifier_sgd.fit(X_train, y_train)

y_pred = power_classifier_sgd.predict(X_test)

power_classifier_sgd_score = accuracy_score(y_test[:,:col_min]  ,y_pred[:,:col_min])

print("Accuracy mean score for Powered SGD classifier : {0:1.2F} %".format(power_classifier_sgd_score*100))

dict_cls_score['Powered SGD'] = power_classifier_sgd_score

In [ ]:
import p5_util
file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_cls_score['GSCV SGD Regression'] = gscv_sgd_cls.best_score_
p5_util.object_dump(dict_cls_score, file_name)

## <font color='blus'>4.13. Classifiers results</font>

In [ ]:
import pandas as pd
df_result = pd.DataFrame.from_dict( dict_cls_score, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result

In [ ]:
import p6_util_plot
title = "Classifiers accuracy"
p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False)

In [ ]:
dict_benchmark_result = dict()
dict_benchmark_result['TFIDF(1,2)']=8.0/100
dict_benchmark_result['BOW']=8.5/100
dict_benchmark_result['TFIDF(1,1)']=12/100
dict_benchmark_result['Kmeans']=16/100
dict_benchmark_result['LDA-TFIDF']=22/100
dict_benchmark_result['LDA-BOW']=30/100
dict_benchmark_result['W2VEC']=24/100
dict_benchmark_result['MNB']=2/100
dict_benchmark_result['SGD']=4/100
dict_benchmark_result['Bernoulli NB']=20/100
dict_benchmark_result['Logistic Reg.']=25/100
dict_benchmark_result['Chained MNB']=0./100



In [ ]:
import pandas as pd
df_result = pd.DataFrame.from_dict( dict_benchmark_result, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result

In [ ]:
import p6_util_plot
title = "Classifiers accuracy"
p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False)